In [1]:
!pip install openai-whisper moviepy pillow diffusers transformers torch accelerate safetensors


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 18.1 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.8 MB/s eta 0:00:000:00:0100:01
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803405 sha256=9b6cea0cffad78184e8ddbdf76fa56371fdef298621d1946bba84b6a9f0354f8
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


In [6]:
import whisper
from moviepy.editor import ImageClip, AudioFileClip, concatenate_videoclips
from PIL import Image
from diffusers import StableDiffusionPipeline
import torch
import os

# ========== SETUP MODELS ==========

print("Loading Whisper model...")
whisper_model = whisper.load_model("base")

print("Loading Stable Diffusion...")
pipe = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    safety_checker=None
).to("cuda" if torch.cuda.is_available() else "cpu")

# ========== FUNCTIONS ==========

def transcribe_audio(audio_path):
    print(f"Transcribing: {audio_path}")
    result = whisper_model.transcribe(audio_path)
    return result["segments"]

def get_visual_for_text(prompt, idx):
    short_prompt = "cute children's storybook illustration hare and tortoise, soft pastel colors, warm lighting, simple shapes, " + prompt.strip().split('.')[0][:100]
    print(f"[Gen] Creating image for: {short_prompt}")
    image = pipe(short_prompt).images[0]
    path = f"frame_{idx}.jpg"
    image.save(path)
    return path

def create_video(audio_path, segments):
    audio_clip = AudioFileClip(audio_path)
    visuals = []

    for idx, seg in enumerate(segments):
        print(f"[{idx+1}/{len(segments)}] {seg['text']}")
        img_path = get_visual_for_text(seg['text'], idx)
        duration = seg['end'] - seg['start']
        clip = ImageClip(img_path).set_duration(duration)
        visuals.append(clip)

    video = concatenate_videoclips(visuals, method="compose")
    video = video.set_audio(audio_clip)
    video.write_videofile("final_story_video.mp4", fps=24)

def process_audio_story(audio_file):
    segments = transcribe_audio(audio_file)
    create_video(audio_file, segments)

# ========== RUN MAIN ==========

if __name__ == "__main__":
    audio_input_path = "/kaggle/input/shwhwjhj/final_output_1 (2).wav"  # 👈 drop your audio file here
    process_audio_story(audio_input_path)


Loading Whisper model...
Loading Stable Diffusion...


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


Transcribing: /kaggle/input/shwhwjhj/final_output_1 (2).wav
[1/9]  Smugly to the crowd, I bet I could outpace any one of you in a race, especially you
[Gen] Creating image for: cute children's storybook illustration hare and tortoise, soft pastel colors, warm lighting, simple shapes, Smugly to the crowd, I bet I could outpace any one of you in a race, especially you


  0%|          | 0/50 [00:00<?, ?it/s]

...

[9/9]  Her confidence can lead to one's downfall, but slow and steady indeed wins the race.
[Gen] Creating image for: cute children's storybook illustration hare and tortoise, soft pastel colors, warm lighting, simple shapes, Her confidence can lead to one's downfall, but slow and steady indeed wins the race


  0%|          | 0/50 [00:00<?, ?it/s]

Moviepy - Building video final_story_video.mp4.
MoviePy - Writing audio in final_story_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video final_story_video.mp4



Moviepy - Done !
Moviepy - video ready final_story_video.mp4


In [7]:
from IPython.display import Video

Video("/kaggle/working/final_story_video.mp4", embed=True)
